In [5]:
# generic
import pandas as pd
import os
import numpy as np
import scipy as sp

# geospatial plotting
import cartopy
import cartopy.crs as ccrs 
import cartopy.feature as cfeature

# display
from IPython.display import Image
from cartopy.io.img_tiles import GoogleTiles

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# handling netCDFs
import xclim as xc
import xarray as xr

In [7]:
# open a netCDF downloaded from the curl script
xr.load_dataset('project-data/TMSO2/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.001.cam.h0.TMSO2.203501-206912.nc')

ValueError: did not find a match in any of xarray's currently installed IO backends ['scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [1]:
# parse the correct netCDF for so4_a1DDF
# try to get a time seires for deposition
# compare this to some other sulfur fluxes
# see if this is high, low, etc.

In [ ]:
# xr.load_dataset(!!)